# Remaining Useful Life (RUL) Prediction for a Single Engine

This notebook demonstrates how to use a pre-trained LSTM model to predict the Remaining Useful Life (RUL) for a single turbofan engine's sensor data. It replicates the necessary preprocessing steps from the original training pipeline to ensure consistency.

## 1. Import Libraries and Load Pre-trained Assets

We'll start by importing all the necessary libraries and loading the pre-trained LSTM model and the scalers (for features and RUL) that were saved during the model training phase in `PMIE.ipynb`.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib
from sklearn.preprocessing import MinMaxScaler
%run custom_functions.py # Runs the whole custom_function to get all functions/classes in the notebook

# --- Load the pre-trained model and scalers ---
try:
    LTSM_model = tf.keras.models.load_model('model/final_lstm_model.keras') # Or 'best_model.keras'
    print("Successfully loaded LSTM model.")
except Exception as e:
    print(f"Error loading model: {e}. Make sure 'final_lstm_model.keras' (or 'best_model.keras') is in the correct directory.")
    # Fallback to .h5 if .keras fails, though .keras is preferred
    try:
        LTSM_model = tf.keras.models.load_model('final_lstm_model.h5')
        print("Successfully loaded LSTM model from .h5 fallback.")
    except Exception as e_h5:
        print(f"Error loading .h5 model: {e_h5}. Please ensure your model file exists.")
        LTSM_model = None # Set to None to prevent further errors

try:
    feature_scaler = joblib.load('feature_scaler.pkl')
    rul_scaler = joblib.load('rul_scaler.pkl')
    print("Successfully loaded feature and RUL scalers.")
except Exception as e:
    print(f"Error loading scalers: {e}. Make sure 'feature_scaler.pkl' and 'rul_scaler.pkl' are in the correct directory.")
    feature_scaler = None
    rul_scaler = None

# Define the sequence length (hyperparameter) - must match training
sequence_length = 30

if LTSM_model:
    print("\nModel Summary:")
    LTSM_model.summary()
else:
    print("\nModel not loaded. Cannot show summary or proceed with prediction.")

Successfully loaded LSTM model.
Successfully loaded feature and RUL scalers.

Model Summary:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_4 (GRU)                     │ (None, 30, 416)        │       597,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 30, 416)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 30, 128)        │       185,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 30, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 30, 256)        │       198,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 30, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_7 (GRU)                     │ (None, 288)            │       471,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           289 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,359,173 (16.63 MB)

 Trainable params: 1,453,057 (5.54 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,906,116 (11.09 MB)

## 2. Load and Preprocess Single Engine Data

We will load the `test_FD002.txt` dataset and select data for a specific `engine_id`. This data will then undergo the same preprocessing steps as the training data: dropping constant columns, scaling, and generating rolling features.

In [10]:
# Load the raw test data
try:
    test_df_raw = import_data('test/test_FD002.txt')
    print("Successfully loaded raw test data.")
except Exception as e:
    print(f"Error loading 'test_FD002.txt': {e}. Please ensure the file is in the correct directory.")
    test_df_raw = pd.DataFrame() # Create an empty DataFrame to prevent further errors

if not test_df_raw.empty:
    # Choose an engine ID for prediction
    # You can change this to any engine ID present in your test_FD003.txt
    # For example, engine 10 has fewer cycles, which might trigger a warning.
    target_engine_id = 141 # Example: Engine ID 1

    # Extract data for the chosen engine
    single_engine_df = test_df_raw[test_df_raw['engine_id'] == target_engine_id].copy()

    if single_engine_df.empty:
        print(f"Error: No data found for Engine ID {target_engine_id}. Please choose a valid engine ID.")
    else:
        print(f"\nData for Engine ID {target_engine_id} (first 5 rows):\n", single_engine_df.head())
        print(f"Total cycles for Engine ID {target_engine_id}: {len(single_engine_df)}")

        # Define feature columns (must match the features used during training)
        # This list should be exactly what `feature_cols` was in PMIE.ipynb after dropping const_col
        # and before adding rolling features.
        initial_feature_cols = [col for col in single_engine_df.columns if col not in ['engine_id']]

        # Apply feature scaling using the loaded feature_scaler
        if feature_scaler:
            single_engine_df[initial_feature_cols] = feature_scaler.transform(single_engine_df[initial_feature_cols])
            print("\nFeatures scaled for single engine data.")
        else:
            print("\nFeature scaler not loaded. Skipping feature scaling.")

        # --- Generate Rolling Features (must match training parameters) ---
        window_size = 30 # Must match the window_size used in training
        selected_sensors = [col for col in initial_feature_cols if col not in ['op_setting_1','op_setting_2','op_setting_3','cycle']]

        print(f"\nGenerating rolling features with window size {window_size}...")

        for sensor in selected_sensors:
            # Rolling mean
            single_engine_df[f'{sensor}_rolling_mean_{window_size}'] = single_engine_df[sensor].rolling(window=window_size, min_periods=1).mean()
            # Rolling standard deviation
            single_engine_df[f'{sensor}_rolling_std_{window_size}'] = single_engine_df[sensor].rolling(window=window_size, min_periods=1).std()

        # Handle NaNs in rolling features (bfill then ffill, as in PMIE.ipynb)
        for sensor in selected_sensors:
            single_engine_df[f'{sensor}_rolling_std_{window_size}'] = single_engine_df[f'{sensor}_rolling_std_{window_size}'].bfill().ffill()

        # Update feature_cols to include all raw and rolling features
        # This list should now contain all 48 features that the model expects
        final_feature_cols = [col for col in single_engine_df.columns if col not in ['engine_id']]

        print(f"\nSingle engine data after rolling features (last 5 rows):\n", single_engine_df.tail())
        print(f"Total features for prediction: {len(final_feature_cols)}")
        print(f"Expected number of features by model: {LTSM_model.input_shape[2] if LTSM_model else 'N/A'}")
else:
    print("Cannot proceed without raw test data.")


Successfully loaded raw test data.

Data for Engine ID 141 (first 5 rows):
        engine_id  cycle  op_setting_1  op_setting_2  op_setting_3  sensor_1  \
19011        141      1       20.0046        0.7014         100.0    491.19   
19012        141      2        0.0002        0.0015         100.0    518.67   
19013        141      3       25.0010        0.6200          60.0    462.54   
19014        141      4       10.0026        0.2500         100.0    489.05   
19015        141      5       10.0020        0.2500         100.0    489.05   

       sensor_2  sensor_3  sensor_4  sensor_5  ...  sensor_11  sensor_12  \
19011    607.12   1483.99   1250.80      9.35  ...      44.34     315.66   
19012    642.92   1589.91   1406.07     14.62  ...      47.25     522.19   
19013    536.74   1264.53   1034.97      7.05  ...      36.80     164.84   
19014    604.63   1508.43   1304.44     10.52  ...      45.22     371.88   
19015    604.31   1497.23   1304.99     10.52  ...      45.15     371

## 3. Prepare Sequence for Prediction

Now, we'll use the `create_single_last_sequence` function to extract the final sequence of data for prediction. This sequence must have the correct 3D shape `(1, sequence_length, num_features)`.

In [11]:
if not single_engine_df.empty and LTSM_model and feature_scaler and rul_scaler:
    # Generate the single sequence for prediction
    # The function returns a 2D array (sequence_length, num_features) or None
    single_sequence = create_single_last_sequence(single_engine_df, sequence_length, final_feature_cols)

    if single_sequence is not None:
        # Reshape for the model: (1, sequence_length, num_features)
        # The model expects a batch dimension, even for a single sample.
        input_for_prediction = np.expand_dims(single_sequence, axis=0)

        print(f"\nShape of input for prediction: {input_for_prediction.shape}")
        print(f"Expected model input shape: {LTSM_model.input_shape}")

        if input_for_prediction.shape[1:] == LTSM_model.input_shape[1:]:
            print("Input shape matches model's expected input shape. Proceeding with prediction.")
        else:
            print("\nERROR: Input shape does NOT match model's expected input shape. Check feature engineering and sequence generation.")
            input_for_prediction = None # Prevent prediction if shape is wrong
    else:
        print("\nCould not generate a valid sequence for prediction (engine data too short).")
        input_for_prediction = None
else:
    print("\nCannot prepare sequence. Ensure data, model, and scalers are loaded correctly.")
    input_for_prediction = None


Shape of input for prediction: (1, 30, 61)
Expected model input shape: (None, 30, 61)
Input shape matches model's expected input shape. Proceeding with prediction.


## 4. Make Prediction and Inverse Transform

Finally, we'll use the loaded LSTM model to make a prediction on the prepared sequence and then inverse transform the result to get the RUL in its original scale.

In [12]:
if input_for_prediction is not None:
    # Make the prediction
    predicted_rul_scaled = LTSM_model.predict(input_for_prediction)

    # Inverse transform the prediction to get the real RUL value
    predicted_rul_original_scale = rul_scaler.inverse_transform(predicted_rul_scaled)

    print(f"\nPredicted RUL (scaled): {predicted_rul_scaled[0][0]:.4f}")
    print(f"Predicted RUL (original scale) for Engine ID {target_engine_id}: {predicted_rul_original_scale[0][0]:.2f} cycles")
else:
    print("\nPrediction skipped due to issues with data preparation.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

Predicted RUL (scaled): 0.2077
Predicted RUL (original scale) for Engine ID 141: 112.57 cycles


I0000 00:00:1751912247.851198    4979 cuda_dnn.cc:529] Loaded cuDNN version 90300
